In [ ]:
from fastai.text.all import *

# Data

In [ ]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

(#2) [Path('/home/renato/.fastai/data/human_numbers/valid.txt'),Path('/home/renato/.fastai/data/human_numbers/train.txt')]

In [ ]:
x_train = [o.strip() for o in (path/'train.txt').readlines()]
y_train = list(range(1, len(x_train)+1))

In [ ]:
x_train[:5], y_train[:5]

(['one', 'two', 'three', 'four', 'five'], [1, 2, 3, 4, 5])

In [ ]:
x_train[-3:], y_train[-3:]

(['seven thousand nine hundred ninety seven',
  'seven thousand nine hundred ninety eight',
  'seven thousand nine hundred ninety nine'],
 [7997, 7998, 7999])

In [ ]:
x_valid = [o.strip() for o in (path/'valid.txt').readlines()]
y_valid = list(range(8001, len(x_valid)+8001))

In [ ]:
x_valid[:3], y_valid[:3]

(['eight thousand one', 'eight thousand two', 'eight thousand three'],
 [8001, 8002, 8003])

In [ ]:
x_valid[-3:], y_valid[-3:]

(['nine thousand nine hundred ninety seven',
  'nine thousand nine hundred ninety eight',
  'nine thousand nine hundred ninety nine'],
 [9997, 9998, 9999])

In [ ]:
df = pd.concat([    
    pd.DataFrame({'text': x_train, 'label': y_train, 'is_valid': False}),
    pd.DataFrame({'text': x_valid, 'label': y_valid, 'is_valid': True}),
])
df.sample(3)

,text,label,is_valid
3360,three thousand three hundred sixty one,3361,False
819,eight thousand eight hundred twenty,8820,True
1022,nine thousand twenty three,9023,True


In [ ]:
dls = TextDataLoaders.from_df(df, text_col='text', label_col='label', valid_col='is_valid',
                              y_block=RegressionBlock)
dls.show_batch(max_n=3)

,text,text_
0,xxbos one thousand one hundred twenty one,1121.0
1,xxbos five thousand four hundred sixty five,5465.0
2,xxbos four thousand six hundred forty seven,4647.0


In [ ]:
dls.train_ds

(#7999) [(TensorText([ 2, 11]), tensor(1.)),(TensorText([ 2, 12]), tensor(2.)),(TensorText([ 2, 13]), tensor(3.)),(TensorText([ 2, 14]), tensor(4.)),(TensorText([ 2, 15]), tensor(5.)),(TensorText([ 2, 16]), tensor(6.)),(TensorText([ 2, 17]), tensor(7.)),(TensorText([ 2, 19]), tensor(8.)),(TensorText([ 2, 18]), tensor(9.)),(TensorText([ 2, 28]), tensor(10.))...]

In [ ]:
dls.valid_ds

(#1999) [(TensorText([ 2, 19, 10, 11]), tensor(8001.)),(TensorText([ 2, 19, 10, 12]), tensor(8002.)),(TensorText([ 2, 19, 10, 13]), tensor(8003.)),(TensorText([ 2, 19, 10, 14]), tensor(8004.)),(TensorText([ 2, 19, 10, 15]), tensor(8005.)),(TensorText([ 2, 19, 10, 16]), tensor(8006.)),(TensorText([ 2, 19, 10, 17]), tensor(8007.)),(TensorText([ 2, 19, 10, 19]), tensor(8008.)),(TensorText([ 2, 19, 10, 18]), tensor(8009.)),(TensorText([ 2, 19, 10, 28]), tensor(8010.))...]

In [ ]:
dls.vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 'hundred',
 'thousand',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'nine',
 'eight',
 'twenty',
 'thirty',
 'forty',
 'fifty',
 'sixty',
 'seventy',
 'eighty',
 'ninety',
 'ten',
 'eleven',
 'twelve',
 'thirteen',
 'fourteen',
 'fifteen',
 'sixteen',
 'seventeen',
 'eighteen',
 'nineteen',
 'xxfake',
 'xxfake']

In [ ]:
dls.train_ds[0]

(TensorText([ 2, 11]), tensor(1.))

In [ ]:
dls.vocab[2], dls.vocab[11]

('xxbos', 'one')

In [ ]:
# for xb,yb in progress_bar(dls.train):
#     if xb.eq(1).any().item():
#         print(':D')
#         break

## Model

In [ ]:
dls.train_ds[0]

(TensorText([ 2, 11]), tensor(1.))

In [ ]:
dls.train_ds[-1]

(TensorText([ 2, 17, 10, 18,  9, 27, 18]), tensor(7999.))

In [ ]:
dls.valid_ds[-1]

(TensorText([ 2, 18, 10, 18,  9, 27, 18]), tensor(9999.))

In [ ]:
xb,yb = dls.one_batch()
xb.shape, yb.shape

(torch.Size([64, 7]), torch.Size([64]))

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.embs = nn.Embedding(40, 100, padding_idx=1)
        self.lstm = nn.LSTM(100, 20, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(20, 8),
            nn.Linear(8, 1)
        )
    
    def forward(self, x):
        embs = self.embs(x)
        # print(embs.shape)
        h,_ = self.lstm(embs)
        h = h[:,-1]
        return self.fc(h)
    
model = Model().cuda()

In [ ]:
model

Model(
  (embs): Embedding(40, 100, padding_idx=1)
  (lstm): LSTM(100, 20, batch_first=True)
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=8, bias=True)
    (1): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [ ]:
with torch.no_grad():
    res = model(xb)
res.shape

torch.Size([64, 1])

In [ ]:
learn = Learner(dls, model, loss_func=L1LossFlat())

## Train

In [ ]:
learn.fit_one_cycle(10, 1e-2)

epoch,train_loss,valid_loss,time
0,1829.946289,6219.750000,00:01
1,1442.818359,5308.075684,00:01
2,829.451843,4518.303223,00:01
3,417.108124,4379.394531,00:01
4,230.437637,4466.254395,00:01
5,140.532700,4325.104004,00:01
6,110.421570,4290.753418,00:01
7,89.501289,4353.627441,00:01
8,73.838448,4292.385742,00:01
9,66.335403,4296.185059,00:01


In [ ]:
learn.show_results(0)

,text,text_,text__
0,xxbos one thousand one hundred twenty one,1121.0,"(1217.9658203125,)"
1,xxbos two thousand one hundred seventy five,2175.0,"(2162.40576171875,)"
2,xxbos one thousand four hundred thirty two,1432.0,"(1464.802978515625,)"
3,xxbos six thousand six hundred eighty five,6685.0,"(6689.7373046875,)"
4,xxbos four thousand six hundred thirty six,4636.0,"(4642.24365234375,)"
5,xxbos three thousand two hundred fifty five,3255.0,"(3209.96875,)"
6,xxbos five thousand six hundred eighty three,5683.0,"(5737.9873046875,)"
7,xxbos four thousand one hundred forty two,4142.0,"(4257.59619140625,)"
8,xxbos one thousand two hundred ninety nine,1299.0,"(1294.419921875,)"
